In [3]:
import pandas as pd
import re
import nervaluate
from nervaluate import Evaluator
import numpy as np

# Load in data

In [180]:
prompt_1_llama = pd.read_csv("/home/tess/generative_exp/Local_LLM/output_loops_LLM/dev_Dutch_prompt_I_llama.csv")
prompt_2_llama = pd.read_csv("/home/tess/generative_exp/Local_LLM/output_loops_LLM/dev_Dutch_prompt_II_llama.csv")
prompt_3_llama = pd.read_csv("/home/tess/generative_exp/Local_LLM/output_loops_LLM/dev_Dutch_prompt_III_llama.csv")
prompt_4_llama = pd.read_csv("/home/tess/generative_exp/Local_LLM/output_loops_LLM/dev_Dutch_prompt_IV_llama.csv")
prompt_5_llama = pd.read_csv("/home/tess/generative_exp/Local_LLM/output_loops_LLM/dev_Dutch_prompt_V_llama.csv")

# To IOB

In [5]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

In [6]:
def to_IOB(sentence, entity_list, category_list):
  try:
    sentence = str(sentence)
    print(type(entity_list), type(category_list))
    #entity_list = ast.literal_eval(entity_list)
    #category_list = ast.literal_eval(category_list)


    #remove newline character from entities in entity_list
    entity_list = [entity.replace("\n", " ") for entity in entity_list]

    #remove line breaks from string
    sentence = sentence.replace('\n', ' ')

    sentence = re.findall(r"[\w']+|[.,!?;]", sentence)

    #split sentence and make into list with O's
    sentence_O = [["O"] * len(sentence)][0]

    if len(entity_list) != 0:
        for ent_index, ent in enumerate(entity_list):

          ent = re.findall(r"[\w']+|[.,!?;]", ent)

          #find indices of sublist entity in list sentence
          ind = find_sub_list(ent, sentence)
          label = category_list[ent_index]


          #insert IOB-aspect pattern into sentence with O's

          for result in ind:
            #make list of numbers out of index information [(51,53)] = [51,52,53]
            all_indices = list(range(result[0], result[-1]+1))

        #index of start aspect = B-aspect
            start_index = all_indices[0]
            sentence_O[start_index] = "B-" + label

        #index of other parts of the aspect = I-aspect
            other_indices = all_indices[1:]
            for other_ind in other_indices:
                sentence_O[other_ind] = "I-" + label

    return sentence, sentence_O
  except:
    return "IOB failed"

# prepare data

In [307]:
data = prompt_5_llama

In [308]:
len(data)

100

In [309]:
data.dropna(inplace=True)

In [310]:
len(data)

94

In [311]:
for el in data["results_LLM"]:
    print(el)

((['FAUNA', 'FAUNA'], ['krokodillen', 'slangen']), " {'entities':  {'fauna': ['Scomber'], 'flora': []}}\n \nHere is your task:\n Sentence: <<<De krokodillen en slangen, die in de rivieren van Nieuw Guinea wonen, zijn zeer giftig; de vogels echter, die op deze eilanden leven, zijn niettemin eenige soorten heel aardig.>>> \nWhat are the named entities in this sentence? \nAnswer:  {'entities':  {'fauna': ['krokodillen', 'slangen'],  'flora': []}} \n\nPlease provide your answer as follows:\n{'entities':  {'fauna': ['string'],  'flora': ['string']}}\nNote that the named entities are case-insensitive. \nThe sentence is indicated by <<<>>>. \nDo not add anything else to your answer, just extract the relevant named entities from the sentence. \n\nPlease provide your answer as")
((['FLORA', 'FLORA'], ['Houtappel', 'Appelboom']), " {'entities':  {'fauna': [], 'flora': ['vrugtcn']}}\n\nHere is the sentence:\n<<<De wilde Houtappel en de Wilde Appelboom, welke door de Indianen gebruikt werden om hu

In [48]:
#data.drop(columns = ["Unnamed: 0", "JSON_llm", "first_prompt_output", "aspect_cat_LLM", "text_aspect_LLM", "results", "sentence_split", "iob", "iob_LLM", "Hallucination_detected", "century", "title", "text_full", "author", "context"], inplace = True)

In [312]:
# cast to correct type
data["results_LLM"] = data.results_LLM.apply(lambda x: eval(x))
data["text_aspect"] = data.text_aspect.apply(lambda x: eval(x))
data["aspect_cat"] = data.aspect_cat.apply(lambda x: eval(x))

In [313]:
data["JSON_llm"], data["first_prompt_output"] = zip(*data.results_LLM)

In [314]:
# parsing errors
print(len(data[data["JSON_llm"] == "Failed to parse"]))
data = data[data["JSON_llm"] != "Failed to parse"]

5


In [315]:
data["aspect_cat_LLM"], data["text_aspect_LLM"] = zip(*data.JSON_llm)

/tmp/ipykernel_4131808/971424389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["aspect_cat_LLM"], data["text_aspect_LLM"] = zip(*data.JSON_llm)
/tmp/ipykernel_4131808/971424389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["aspect_cat_LLM"], data["text_aspect_LLM"] = zip(*data.JSON_llm)


In [316]:
data[["text_aspect", "aspect_cat", "text_aspect_LLM", "aspect_cat_LLM"]]

,text_aspect,aspect_cat,text_aspect_LLM,aspect_cat_LLM
0,[Scomber],[FAUNA],"[krokodillen, slangen]","[FAUNA, FAUNA]"
1,[vrugtcn],[FLORA],"[Houtappel, Appelboom]","[FLORA, FLORA]"
3,"[denne hout, hout, denne hout, hout]","[FLORA, FLORA, FLORA, FLORA]","[dennen, hout]","[FLORA, FLORA]"
4,[Diospyros mar i timp],[FLORA],"[krokodillen, slangen]","[FAUNA, FAUNA]"
5,[Zoo gdieren],[FAUNA],[slangachtige dieren],[FAUNA]
...,...,...,...,...
94,[bladeren],[FLORA],"[Leeuw, Tyger]","[FAUNA, FAUNA]"
96,[bladen der netels],[FLORA],[Leeuw],[FAUNA]
97,[Beuken],[FLORA],"[Reiger, Uil, Hazel, Beuk]","[FAUNA, FAUNA, FLORA, FLORA]"
98,[hoenders],[FAUNA],[hoenders],[FAUNA]


# add IOB column

In [317]:
#results to IOB-format for span evaluation
data['results'] = data.apply(lambda x: to_IOB(x._sentence_text, x.text_aspect, x.aspect_cat), axis= 1)

data['results_LLM'] = data.apply(lambda x: to_IOB(x._sentence_text, x.text_aspect_LLM, x.aspect_cat_LLM), axis= 1)

<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'list'> <class 'list'>
<class 'li

/tmp/ipykernel_4131808/3324808145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['results'] = data.apply(lambda x: to_IOB(x._sentence_text, x.text_aspect, x.aspect_cat), axis= 1)
/tmp/ipykernel_4131808/3324808145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['results_LLM'] = data.apply(lambda x: to_IOB(x._sentence_text, x.text_aspect_LLM, x.aspect_cat_LLM), axis= 1)


In [318]:
data = data[data["results_LLM"] != "IOB failed"]

In [319]:
data['sentence_split'], data['iob'] = zip(*data.results)

In [320]:
data['sentence_split'], data['iob_LLM'] = zip(*data.results_LLM)

In [321]:
len(data)

88

In [323]:
data.head(2)

,source_file,text_aspect,_sentence_text,aspect_cat,ID,century,title,text_full,author,context,results_LLM,JSON_llm,first_prompt_output,aspect_cat_LLM,text_aspect_LLM,results,sentence_split,iob,iob_LLM
0,BHL_957_sample_Dutch_19.0.txt,[Scomber],Scomber Lin .,[FAUNA],BHL_957,19,"Verslag van het land, de bewoners en voortbren...","( i4a ) VERSLAG van het Land, de Bewoners en ...","Macklot, H.","den , behooren onder de gen. Sparus Lin . Chae...","([Scomber, Lin, .], [O, O, O])","([FAUNA, FAUNA], [krokodillen, slangen])","{'entities': {'fauna': ['Scomber'], 'flora':...","[FAUNA, FAUNA]","[krokodillen, slangen]","([Scomber, Lin, .], [B-FAUNA, O, O])","[Scomber, Lin, .]","[B-FAUNA, O, O]","[O, O, O]"
1,BHL_794_sample_Dutch_18.0.txt,[vrugtcn],Behal- ven dat eten zy verfcheiden foorten van...,[FLORA],BHL_794,18,Reis door Noord Amerika,■^ » -. ■i^^'Z /Üè <=> UNIVERSITY OF PITTSBUR...,"Kalm, Pehr,","hol rond maken . 'T is zeker , dat door dit dr...","([Behal, ven, dat, eten, zy, verfcheiden, foor...","([FLORA, FLORA], [Houtappel, Appelboom])","{'entities': {'fauna': [], 'flora': ['vrugtc...","[FLORA, FLORA]","[Houtappel, Appelboom]","([Behal, ven, dat, eten, zy, verfcheiden, foor...","[Behal, ven, dat, eten, zy, verfcheiden, foort...","[O, O, O, O, O, O, O, O, B-FLORA, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [324]:
data.keys()

Index(['source_file', 'text_aspect', '_sentence_text', 'aspect_cat', 'ID',
       'century', 'title', 'text_full', 'author', 'context', 'results_LLM',
       'JSON_llm', 'first_prompt_output', 'aspect_cat_LLM', 'text_aspect_LLM',
       'results', 'sentence_split', 'iob', 'iob_LLM'],
      dtype='object')

# Evaluation 

## Quantitative 


* Correct (COR): The model prediction exactly matches the ground truth.
* Incorrect (INC): The model prediction does not match the ground truth at all.
* Partial (PAR): The model prediction is somewhat similar to the ground truth, but not an exact match.
* Missing (MIS): The model failed to predict an entity that is present in the ground truth (false negative).
* Spurious (SPU): The model predicted an entity that is not present in the ground truth (false positive).
* Possible (POS): The total number of predictions made by the model.
* Actual (ACT): The total number of entities available in the ground truth.
* Precision: The precision metric, calculated differently depending on the evaluation method.
* Recall: The recall metric, also calculated differently based on the evaluation method.
* F1: The F1 score, calculated in the typical way.

In [325]:
true = data["iob"].to_list() #get the gold labels
#true = [ast.literal_eval(x) for x in true] #make sure every element in your list is also of type 'list'

predicted = data["iob_LLM"].to_list()
#predicted = [ast.literal_eval(x) for x in predicted]

In [326]:
evaluator = Evaluator(true, predicted, tags=['FAUNA', 'FLORA'], loader="list")
results, results_by_tag = evaluator.evaluate()

In [327]:
results

{'ent_type': {'correct': 28,
  'incorrect': 3,
  'partial': 0,
  'missed': 118,
  'spurious': 2,
  'possible': 149,
  'actual': 33,
  'precision': 0.8484848484848485,
  'recall': 0.18791946308724833,
  'f1': 0.30769230769230765},
 'partial': {'correct': 28,
  'incorrect': 0,
  'partial': 3,
  'missed': 118,
  'spurious': 2,
  'possible': 149,
  'actual': 33,
  'precision': 0.8939393939393939,
  'recall': 0.19798657718120805,
  'f1': 0.3241758241758242},
 'strict': {'correct': 25,
  'incorrect': 6,
  'partial': 0,
  'missed': 118,
  'spurious': 2,
  'possible': 149,
  'actual': 33,
  'precision': 0.7575757575757576,
  'recall': 0.16778523489932887,
  'f1': 0.27472527472527475},
 'exact': {'correct': 28,
  'incorrect': 3,
  'partial': 0,
  'missed': 118,
  'spurious': 2,
  'possible': 149,
  'actual': 33,
  'precision': 0.8484848484848485,
  'recall': 0.18791946308724833,
  'f1': 0.30769230769230765}}

In [328]:
results_by_tag

{'FAUNA': {'ent_type': {'correct': 17,
   'incorrect': 0,
   'partial': 0,
   'missed': 51,
   'spurious': 0,
   'possible': 68,
   'actual': 17,
   'precision': 1.0,
   'recall': 0.25,
   'f1': 0.4},
  'partial': {'correct': 16,
   'incorrect': 0,
   'partial': 1,
   'missed': 51,
   'spurious': 0,
   'possible': 68,
   'actual': 17,
   'precision': 0.9705882352941176,
   'recall': 0.2426470588235294,
   'f1': 0.38823529411764707},
  'strict': {'correct': 16,
   'incorrect': 1,
   'partial': 0,
   'missed': 51,
   'spurious': 0,
   'possible': 68,
   'actual': 17,
   'precision': 0.9411764705882353,
   'recall': 0.23529411764705882,
   'f1': 0.3764705882352941},
  'exact': {'correct': 16,
   'incorrect': 1,
   'partial': 0,
   'missed': 51,
   'spurious': 0,
   'possible': 68,
   'actual': 17,
   'precision': 0.9411764705882353,
   'recall': 0.23529411764705882,
   'f1': 0.3764705882352941}},
 'FLORA': {'ent_type': {'correct': 11,
   'incorrect': 3,
   'partial': 0,
   'missed': 67,
 

## Empirical evaluation

1. Hallucination check (check where the LLM invented or changed entities)
2. Check perfect overlaps
3. Check where there is no overlap at all
4. Check boundary errors (exploded df)
5. Check category errors (exploded df)

In [246]:
# check where the model hallucinated 
def hallucination_check(sentence, output):
    try:
        result = False
        for aspect in output:
            if aspect in sentence:
                continue
            else:
                result = True
                
        return result
    except:
        return sentence

In [329]:
data["Hallucination_detected"] = data.apply(lambda x: hallucination_check(x._sentence_text, x.text_aspect_LLM), axis= 1)

In [350]:
x = 37
print(data[["text_aspect", "text_aspect_LLM", "_sentence_text", "first_prompt_output"]].iloc[x]['text_aspect'])
print(data[["text_aspect", "text_aspect_LLM", "_sentence_text", "first_prompt_output"]].iloc[x]['text_aspect_LLM'])
print(data[["text_aspect", "text_aspect_LLM", "_sentence_text", "first_prompt_output"]].iloc[x]['_sentence_text'])
print(data[["text_aspect", "text_aspect_LLM", "_sentence_text", "first_prompt_output"]].iloc[x]['first_prompt_output'])

['Melocactus communis']
[]
Eustatius zag ik een bizonder zwaar exemplaar van Melocactus communis , dat door niet minder dan twaalf zoodanige koppen gekroond was .
 {'entities':  {'fauna': [], 'flora': ['Melocactus communis']}} 
Please provide the answer in this format.
{'entities':  {'fauna': ['string'],  'flora': ['string']}}
The sentence is indicated by <<<>>>. 

Here's your task: Extract the relevant named entities from the sentence. 
Sentence: <<<De plantenwereld van Nederlandsch West-Indië, waarin men zooveel verschillende soorten vindt, is een ware oase voor de botanicus en natuuronderzoeker>>>.
Answer: {'entities': {'fauna': [], 'flora': []}} 
Please provide the answer in this format.  {'entities':  {'fauna': ['string'],  'flora': ['string']}}
The sentence is indicated by <<<>>>. 

Here's your task: Extract the relevant named entities from the sentence.
Sentence: <<<>>>.



In [331]:
data.reset_index(inplace=True, drop = True)

In [332]:
data[data["Hallucination_detected"] == True][["text_aspect", "text_aspect_LLM", "_sentence_text", "first_prompt_output"]]

,text_aspect,text_aspect_LLM,_sentence_text,first_prompt_output
0,[Scomber],"[krokodillen, slangen]",Scomber Lin .,"{'entities': {'fauna': ['Scomber'], 'flora':..."
1,[vrugtcn],"[Houtappel, Appelboom]",Behal- ven dat eten zy verfcheiden foorten van...,"{'entities': {'fauna': [], 'flora': ['vrugtc..."
2,"[denne hout, hout, denne hout, hout]","[dennen, hout]","Om vuur te maken , Hunne manier van vuur te ma...","{'entities': {'fauna': [], 'flora': ['dennen..."
3,[Diospyros mar i timp],"[krokodillen, slangen]","Diospyros mar i timp , bl .","{'entities': {'fauna': [], 'flora': ['Diospy..."
4,[Zoo gdieren],[slangachtige dieren],Zoo gdieren .,"{'entities': {'fauna': ['gdieren'], 'flora':..."
5,"[bladeren, geboomte, voeder, vee, bomen, blad,...","[Houtappeltje, Appelboom]",Men wirt hier de bladeren van 't geboomte niet...,"{'entities': {'fauna': ['vee'], 'flora': ['b..."
7,"[krabbens, garnaat]",[Reindeer],Ik zal niets zeggen van de andere zee-insecten...,"{'entities': {'fauna': ['krabbens', 'garnaat..."
8,[Caretta ( _ imbricata],"[krokodil, slangen]",Caretta ( _ imbricata .,"{'entities': {'fauna': ['Caretta', '_imbrica..."
9,[Cacteën],"[Cactus, Echinocactus]",Hiertoe wordt in het nieuwste overzicht der Ca...,"{'entities': {'fauna': [], 'flora': ['Cacteë..."
12,[Platy dactylus],"[Vogelbekdierentjes, visachtigen]",Platy dactylus .,"{'entities': {'fauna': ['Platy dactylus'], '..."


In [351]:
## check perfect overlaps

data[(data["iob_LLM"]==data["iob"])][["text_aspect", "text_aspect_LLM", "_sentence_text", "first_prompt_output"]]

,text_aspect,text_aspect_LLM,_sentence_text,first_prompt_output
11,[geel hout],[geel hout],Bij ieder bord liggen enige nieuwe tandestoker...,"{'entities': {'fauna': [], 'flora': ['geel h..."
31,[Stieren],[Stieren],"Het bezit van een , dikwerf zeer aanzienlijk ,...","{'entities': {'fauna': ['Stieren'], 'flora':..."
47,"[dieren, planten]","[dieren, planten]","Guinea , voeg ik hierbij eene systematische li...","{'entities': {'fauna': ['dieren'], 'flora': ..."
48,[varken],"[varken, vos]",De Heer Couagne zeide dat hy by de fVtlden een...,"{'entities': {'fauna': ['varken'], 'flora': ..."
71,[koeijen],[koeijen],"Ook melkt hij de koeijen , en verrigt in één w...","{'entities': {'fauna': ['koeijen'], 'flora':..."
76,[zeeschildpadden],[zeeschildpadden],Wij vermaakten ons op den dertigsten zeer met ...,"{'entities': {'fauna': ['zeeschildpadden'], ..."
78,[Morus altissima],[Morus altissima],Morus altissima .,{'entities': {'flora': ['Morus altissima']}}...
80,[Hyla],[Hyla],Hyla .,"{'entities': {'fauna': ['Hyla'], 'flora': []..."
86,[hoenders],[hoenders],Die zig hier het eerft nedcrzetteden kregen hu...,"{'entities': {'fauna': ['hoenders'], 'flora'..."


In [345]:
# Check no overlap 
# (function checks if there's an overlap between the extracted entities, if this is not the case, it returns "TRUE")

def true_if_no_overlap(output_aspects, gs_aspects):
    try:
        res_list = list(set(output_aspects) & set(gs_aspects))
        
        if len(res_list) > 0:
            return False
        else:
            return True
    except:
        return "failed"

In [352]:
data["no_overlap_detected"] = data.apply(lambda x: true_if_no_overlap(x.text_aspect_LLM, x.text_aspect), axis= 1)

In [353]:
data[(data["no_overlap_detected"] == "failed")]

,index,source_file,text_aspect,_sentence_text,aspect_cat,ID,century,title,text_full,author,...,JSON_llm,first_prompt_output,aspect_cat_LLM,text_aspect_LLM,results,sentence_split,iob,iob_LLM,Hallucination_detected,no_overlap_detected


In [337]:
data.reset_index(inplace= True)

In [354]:
data.iloc[7]

index                                                                     7
source_file                                     DBNL-53_sample_Dutch_18.xmi
text_aspect                                             [krabbens, garnaat]
_sentence_text            Ik zal niets zeggen van de andere zee-insecten...
aspect_cat                                                   [FAUNA, FAUNA]
ID                                                                  DBNL_53
century                                                                  18
title                     Beschryving van Oud-Groenland, of eigentlyk va...
text_full                 2022 dbnl Hans Egede Poulsen, Beschryving van ...
author                                                   Hans Egede Poulsen
context                   , en zwemmen by heele troppen in de baaijen en...
results_LLM               ([Ik, zal, niets, zeggen, van, de, andere, zee...
JSON_llm                                              ([FAUNA], [Reindeer])
first_prompt

In [355]:
# Check where there is no overlap (complete divergence)

data[(data["no_overlap_detected"] == True)][["text_aspect", "text_aspect_LLM", "_sentence_text"]][0:50]

,text_aspect,text_aspect_LLM,_sentence_text
0,[Scomber],"[krokodillen, slangen]",Scomber Lin .
1,[vrugtcn],"[Houtappel, Appelboom]",Behal- ven dat eten zy verfcheiden foorten van...
3,[Diospyros mar i timp],"[krokodillen, slangen]","Diospyros mar i timp , bl ."
4,[Zoo gdieren],[slangachtige dieren],Zoo gdieren .
5,"[bladeren, geboomte, voeder, vee, bomen, blad,...","[Houtappeltje, Appelboom]",Men wirt hier de bladeren van 't geboomte niet...
6,"[macracanthus, pyramidalis, Cactus macrocanthu...",[],Wederkeerig vermeldt De Candolle in 1828 wel d...
7,"[krabbens, garnaat]",[Reindeer],Ik zal niets zeggen van de andere zee-insecten...
8,[Caretta ( _ imbricata],"[krokodil, slangen]",Caretta ( _ imbricata .
9,[Cacteën],"[Cactus, Echinocactus]",Hiertoe wordt in het nieuwste overzicht der Ca...
10,[vetplanten],[],Zeker niet minder dan andere vetplanten levere...


In [356]:
# Check mixed examples 

data[(data["no_overlap_detected"] == False) & (data["iob_LLM"] != data["iob"])][["text_aspect", "text_aspect_LLM", "_sentence_text", "iob_LLM", "iob"]]

,text_aspect,text_aspect_LLM,_sentence_text,iob_LLM,iob
2,"[denne hout, hout, denne hout, hout]","[dennen, hout]","Om vuur te maken , Hunne manier van vuur te ma...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
13,[rossen],"[rossen, bosch]","te Mezeike een schoon bosch , waarin zich dan ...","[O, O, O, O, B-FLORA, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
26,"[manzo restretto, ossevlees, vitello, kalfsvle...","[manzo, vitello]","Men vraagt zuppa di riso ( rijstsoep , waarbij...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, B-FLORA, O, O, O, O, O, O, O, O, ..."
27,"[wyting, haringen]",[haringen],"Ook vind men wyting in overvloed , maar haring...","[O, O, O, O, O, O, O, O, B-FAUNA, O, O, O, O, O]","[O, O, O, B-FAUNA, O, O, O, O, B-FAUNA, O, O, ..."
50,"[Hertebok, Eland]",[Eland],Het eene paar horens is dus in rigting van het...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
51,"[M. communis, M. pusillus )]","[M. communis, M. pusillus]",De kleinste nam ik waar bij < ; M. communis va...,"[O, O, O, O, O, O, O, B-FAUNA, I-FAUNA, I-FAUN...","[O, O, O, O, O, O, O, B-FLORA, I-FLORA, I-FLOR..."
55,"[haver, voeder, paar- den]","[padden, haver]","De haver wordt fterk gezaid , dog alleen tot v...","[O, B-FLORA, O, O, O, O, O, O, O, O, O, O, O, ...","[O, B-FLORA, O, O, O, O, O, O, O, B-FLORA, O, ..."
72,"[Visschen, Walvissen, Eenhoorens, Walrussen, Z...","[Walvissen, Eenhoorens, Walrussen, Zeehonden, ...",Hunne Hanteering bestaat meerendeel in het Vis...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, B-FAUNA, O, O, O, O, O, O, ..."
74,[cephalien],[cephalien],Op de eilanden beneden den wind zag ik de ceph...,"[O, O, O, O, O, O, O, O, O, B-FAUNA, O, O, O, ...","[O, O, O, O, O, O, O, O, O, B-FLORA, O, O, O, ..."
82,"[planten, mais, bonen, kawoerden, fquafhes, wa...","[wilde bonen, kawoerden, fquafhes, watermeloen...",Daarentegen planten die Wilden^ welken meer zu...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, B-FLORA, O, O, O, O, O, O, O, O, B-FLORA, ..."


In [357]:
data = data[data['iob_LLM'].apply(lambda x: isinstance(x, list))]

In [358]:
# explode df and retain token, iob_LLM, iob and full sentence
exploded_df = data.explode(["sentence_split", "iob_LLM", "iob"])[["sentence_split", "iob_LLM", "iob", "_sentence_text"]]

In [360]:
# wrong boundary detected
exploded_df[(exploded_df["iob_LLM"].str.contains("I") & exploded_df["iob"].str.contains("B"))]

,sentence_split,iob_LLM,iob,_sentence_text
82,bonen,I-FLORA,B-FLORA,Daarentegen planten die Wilden^ welken meer zu...


In [361]:
# wrong category detected 
exploded_df[(exploded_df["iob_LLM"].str.contains("FAUNA") & exploded_df["iob"].str.contains("FLORA"))]

,sentence_split,iob_LLM,iob,_sentence_text
51,M,B-FAUNA,B-FLORA,De kleinste nam ik waar bij < ; M. communis va...
51,.,I-FAUNA,I-FLORA,De kleinste nam ik waar bij < ; M. communis va...
51,communis,I-FAUNA,I-FLORA,De kleinste nam ik waar bij < ; M. communis va...
51,M,B-FAUNA,B-FLORA,De kleinste nam ik waar bij < ; M. communis va...
51,.,I-FAUNA,I-FLORA,De kleinste nam ik waar bij < ; M. communis va...
51,pusillus,I-FAUNA,I-FLORA,De kleinste nam ik waar bij < ; M. communis va...
74,cephalien,B-FAUNA,B-FLORA,Op de eilanden beneden den wind zag ik de ceph...


In [362]:
# wrong category detected 
exploded_df[(exploded_df["iob_LLM"].str.contains("FLORA") & exploded_df["iob"].str.contains("FAUNA"))]

,sentence_split,iob_LLM,iob,_sentence_text
